In [ ]:
%pip install riotwatcher

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
from riotwatcher import LolWatcher, ApiError
from google.colab import drive
import pandas as pd
import requests
import json
import time
import pandas as pd
import time
import random
from threading import Thread
from google.colab import files

In [ ]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
key1='RGAPI-facc71f1-d46a-4e14-9e21-c45cec8600c2'
Ws=LolWatcher(key1)
region = 'na1'

In [ ]:

summID_list = []


def summ_ID_puller(region, division, tier, page, key):
    global summID_list, request_count
    url_pull = "https://{}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{}/{}/?page={}&api_key={}".format(region, division, tier, page, key)

    response = requests.get(url_pull)


    if response.status_code == 403:
        print("Error 403: Forbidden - Check API key and permissions.")
        return

    profile_list = response.json()


    num_profiles = len(profile_list)
    for profile in range(num_profiles):
        if 'summonerId' in profile_list[profile]:
            summID_list.append(profile_list[profile]['summonerId'])
        else:
            print("KeyError: 'summonerId' not found in profile", profile_list[profile])

    request_count += 1
    if request_count % 20 == 0:
        print("Sleeping for 1 second to respect rate limit...")
        time.sleep(1.5) # Sleep for 1 second after every 20 requests
    if request_count % 100 == 0:
        print("Sleeping for 2 minutes to respect rate limit...")
        time.sleep(120)  # Sleep for 2 minutes after every 100 requests

for tier in ["I", "II", "III", "IV"]:
    for page in range(1, 24):
        print("Page: ", page)
        summ_ID_puller(region, "DIAMOND", tier, page, key1)
        if len(summID_list) % 200 == 0:
          df = pd.DataFrame(summID_list, columns=["Summoner ID"])
          df.to_csv('summID.csv', mode='w')
        print(len(summID_list))




Page:  1
205
Page:  2
410
Page:  3
615
Page:  4
820
Page:  5
1025
Page:  6
1230
Page:  7
1435
Page:  8
1640
Page:  9
1845
Page:  10
2050
Page:  11
2255
Page:  12
2460
Page:  13
2665
Page:  14
2870
Page:  15
3075
Page:  16
3280
Page:  17
3485
Page:  18
3690
Page:  19
3895
Page:  20
Sleeping for 1 second to respect rate limit...
4100
Page:  21
4305
Page:  22
4510
Page:  23
4715
Page:  1
4920
Page:  2
5125
Page:  3
5330
Page:  4
5535
Page:  5
5740
Page:  6
5945
Page:  7
6150
Page:  8
6355
Page:  9
6560
Page:  10
6765
Page:  11
6970
Page:  12
7175
Page:  13
7380
Page:  14
7585
Page:  15
7790
Page:  16
7995
Page:  17
Sleeping for 1 second to respect rate limit...
8200
Page:  18
8405
Page:  19
8610
Page:  20
8815
Page:  21
9020
Page:  22
9225
Page:  23
9430
Page:  1
9635
Page:  2
9840
Page:  3
10045
Page:  4
10250
Page:  5
10455
Page:  6
10660
Page:  7
10865
Page:  8
11070
Page:  9
11275
Page:  10
11480
Page:  11
11685
Page:  12
11890
Page:  13
12095
Page:  14
Sleeping for 1 second to respec

In [ ]:
random.shuffle(summID_list)
df = pd.DataFrame(summID_list,columns = ["Summoner ID"])
df.to_csv('summID.csv',mode = 'w')

In [ ]:
summoner_IDs = pd.read_csv("summID.csv")
summID_list = summoner_IDs["Summoner ID"]

PUUIDs_list = []
request_count = 0

def PUUID_puller(region, summID, k):
    global PUUIDs_list, request_count
    url_acct_pull = "https://{}.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}".format(region, summID, k)


    response = requests.get(url_acct_pull)

    if response.status_code == 403:
        print("Error 403: Forbidden - Check your API key and permissions.")
        return
    elif response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return

    account_info = response.json()
    try:
        PUUIDs_list.append(account_info["puuid"])
    except KeyError:
        print("KeyError: 'puuid' not found in response", account_info)
    request_count += 1
    if request_count % 20 == 0:
        print("Sleeping for 1 second to respect rate limit...")
        time.sleep(1)  # Sleep for 1 second after every 20 requests
    if request_count % 100 == 0:
        print("Sleeping for 2 minutes to respect rate limit...")
        time.sleep(120)  # Sleep for 2 minutes after every 100 requests

for i in range(10000, 15000):
    if i >= len(summID_list):
        break
    time.sleep(1.5)
    print("PUUID list length:", len(PUUIDs_list))

    try:
        PUUID_puller(region, summID_list[i], key1)
    except Exception as e:
        print(f"Exception occurred: {e}")

    if len(PUUIDs_list) % 1000 == 0:
        df = pd.DataFrame(PUUIDs_list, columns=["PUUID"])
        df.to_csv('PUUIDs.csv', mode='w')
        break

PUUID list length: 0
PUUID list length: 1
PUUID list length: 2
PUUID list length: 3
PUUID list length: 4
PUUID list length: 5
PUUID list length: 6
PUUID list length: 7
PUUID list length: 8
PUUID list length: 9
PUUID list length: 10
PUUID list length: 11
PUUID list length: 12
PUUID list length: 13
PUUID list length: 14
PUUID list length: 15
PUUID list length: 16
PUUID list length: 17
PUUID list length: 18
PUUID list length: 19
Sleeping for 1 second to respect rate limit...
PUUID list length: 20
PUUID list length: 21
PUUID list length: 22
PUUID list length: 23
PUUID list length: 24
PUUID list length: 25
PUUID list length: 26
PUUID list length: 27
PUUID list length: 28
PUUID list length: 29
PUUID list length: 30
PUUID list length: 31
PUUID list length: 32
PUUID list length: 33
PUUID list length: 34
PUUID list length: 35
PUUID list length: 36
PUUID list length: 37
PUUID list length: 38
PUUID list length: 39
Sleeping for 1 second to respect rate limit...
PUUID list length: 40
PUUID list len

In [ ]:
PUUIDs = pd.read_csv("PUUIDs.csv")
PUUIDs_list = PUUIDs["PUUID"]

match_list = []

def match_puller(puuid, WS):
  match_record = WS.match.matchlist_by_puuid(region, puuid, queue=420)
  if (len(match_record) > 0):
    for i in range(0, len(match_record)):
      matchID = match_record[i]
      match_list.append(matchID)


for i in range(1, len(PUUIDs_list)):
  time.sleep(1)
  match_puller(PUUIDs_list[i], Ws)
  if len(match_list) % 1000 == 0:
    df = pd.DataFrame(match_list, columns=["MatchId"])
    df.to_csv('MatchId.csv', mode='w')
    break
  print(len(match_list))




20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980


In [ ]:
champion_list_url = "https://ddragon.leagueoflegends.com/cdn/14.15.1/data/en_US/champion.json"
response = requests.get(champion_list_url)
champion_id_to_name = {}
if response.status_code == 200:
    data = response.json()
    champions = data['data']
    champion_id_to_name = {int(champ_info['key']): champ_info['name'] for champ_info in champions.values()}
    print(champion_id_to_name)



{266: 'Aatrox', 103: 'Ahri', 84: 'Akali', 166: 'Akshan', 12: 'Alistar', 32: 'Amumu', 34: 'Anivia', 1: 'Annie', 523: 'Aphelios', 22: 'Ashe', 136: 'Aurelion Sol', 893: 'Aurora', 268: 'Azir', 432: 'Bard', 200: "Bel'Veth", 53: 'Blitzcrank', 63: 'Brand', 201: 'Braum', 233: 'Briar', 51: 'Caitlyn', 164: 'Camille', 69: 'Cassiopeia', 31: "Cho'Gath", 42: 'Corki', 122: 'Darius', 131: 'Diana', 119: 'Draven', 36: 'Dr. Mundo', 245: 'Ekko', 60: 'Elise', 28: 'Evelynn', 81: 'Ezreal', 9: 'Fiddlesticks', 114: 'Fiora', 105: 'Fizz', 3: 'Galio', 41: 'Gangplank', 86: 'Garen', 150: 'Gnar', 79: 'Gragas', 104: 'Graves', 887: 'Gwen', 120: 'Hecarim', 74: 'Heimerdinger', 910: 'Hwei', 420: 'Illaoi', 39: 'Irelia', 427: 'Ivern', 40: 'Janna', 59: 'Jarvan IV', 24: 'Jax', 126: 'Jayce', 202: 'Jhin', 222: 'Jinx', 145: "Kai'Sa", 429: 'Kalista', 43: 'Karma', 30: 'Karthus', 38: 'Kassadin', 55: 'Katarina', 10: 'Kayle', 141: 'Kayn', 85: 'Kennen', 121: "Kha'Zix", 203: 'Kindred', 240: 'Kled', 96: "Kog'Maw", 897: "K'Sante", 7: 'L

In [ ]:
MatchID = pd.read_csv("/content/drive/MyDrive/MatchId.csv")
ID_List = MatchID["MatchId"]
matchChampion_list = []

def getMatchDetail(region, match_id, api_key):
  url_match_detail = "https://{}.api.riotgames.com/lol/match/v5/matches/{}?api_key={}".format("americas",match_id, api_key)
  response = requests.get(url_match_detail)


  if response.status_code != 200:
      print(f"Error {response.status_code}: {response.text}")
      return


  Match = response.json()
  MatchData = Match["info"]
  Team = []
  for detail in MatchData["participants"]:
    champion = detail["championName"]
    teamPosition = detail["teamPosition"]
    if detail["win"]:
      Team.append(teamPosition + "," + champion)

  for team in MatchData["teams"]:
    if team["win"]:
      bans = [champion_id_to_name.get(ban["championId"], "None") if ban else "None" for ban in team.get("bans", [])]
      Team.extend([', '.join(bans)])


  Team.append(round(MatchData["gameDuration"]/(60),2))
  Team.append(MatchData["gameVersion"][:7])

  matchChampion_list.append([str(Data) for Data in Team])

getMatchDetail(region, ID_List[i], key1)
for i in range(0,len(ID_List)):
  time.sleep(1)
  print(len(matchChampion_list))
  getMatchDetail(region, ID_List[i], key1)
  if len(matchChampion_list) % 1000 == 0:
      df = pd.DataFrame(matchChampion_list, columns = ["Champ1", "Champ2", "Champ3", "Champ4", "Champ5","Bans", "Duration in Mins", "Version"])
      df.to_csv("champtionData.csv", mode = "w")
      break






1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
MatchID = pd.read_csv("/content/drive/MyDrive/MatchId.csv")
ID_List = MatchID["MatchId"]
matchChampion_list = []

def getMatchDetail(region, match_id, api_key):
  url_match_detail = "https://{}.api.riotgames.com/lol/match/v5/matches/{}?api_key={}".format("americas",match_id, api_key)
  response = requests.get(url_match_detail)


  if response.status_code != 200:
      print(f"Error {response.status_code}: {response.text}")
      return


  Match = response.json()
  MatchData = Match["info"]
  Team = []
  Team_lost = []
  for detail in MatchData["participants"]:
    champion = detail["championName"]
    teamPosition = detail["teamPosition"]
    if detail["win"]:
      Team.append(teamPosition + "," + champion)
    elif not detail["win"]:
      Team_lost.append(teamPosition + "," + champion)

  for team in MatchData["teams"]:
    if team["win"]:
      bans = [champion_id_to_name.get(ban["championId"], "None") if ban else "None" for ban in team.get("bans", [])]
      Team.extend([', '.join(bans)])


  Team.append(round(MatchData["gameDuration"]/(60),2))
  Team.append(MatchData["gameVersion"][:7])
  Team.extend(Team_lost)

  matchChampion_list.append([str(Data) for Data in Team])


for i in range(0,len(ID_List)):
  time.sleep(1)
  print(len(matchChampion_list))
  getMatchDetail(region, ID_List[i], key1)
  if len(matchChampion_list) % 1000 == 0:
      df = pd.DataFrame(matchChampion_list, columns = ["Champ1", "Champ2", "Champ3", "Champ4", "Champ5","Bans", "Duration in Mins", "Version", "Lost_Champ1", "Lost_Champ2", "Lost_Champ3", "Lost_Champ4", "Lost_Champ5"])
      df.to_csv("chamption_withLost_Data.csv", mode = "w")
      break







0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27